In [1]:
import torch
import numpy as np

In [2]:
from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D, Trainer1D, Dataset1D

/home/kyle/anaconda3/envs/local_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [95]:
model = Unet1D(
    dim = 192,
    dim_mults = (1, 2, 4, 8),
    channels = 1
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'UNet parameters: {trainable_params:,}')

UNet parameters: 127,381,633


In [96]:
diffusion = GaussianDiffusion1D(
    model,
    seq_length = 4096,
    timesteps = 1000,
    objective = 'pred_noise'
)
trainable_params = sum(p.numel() for p in diffusion.parameters() if p.requires_grad)
print(f'diffusion parameters: {trainable_params:,}')

diffusion parameters: 127,381,633


In [97]:

training_seq = torch.rand(64, 1, 4096).float()
loss = diffusion(training_seq)

: 

In [ ]:
loss

tensor(1.1252, grad_fn=<MeanBackward0>)

In [ ]:
data_dir = '/home/kyle/Desktop/DataHub/Defense_DM3/spectra/'

In [ ]:
from data.defense import data_loading 


train_dataset = data_loading.build_dataset(
                    dataset_dir_path=data_dir,
                    input_dim=88300,
                    train_val_or_test='train',
                    return_labels=False
                    )
eval_dataset = data_loading.build_dataset(
                dataset_dir_path=data_dir,
                input_dim=88300,
                train_val_or_test='test',
                return_labels=False
                )

 building dataset
    train
        torch.Size([504, 4096])
        torch.Size([1200, 4096])
        torch.Size([1200, 4096])
        torch.Size([1120, 4096])
        torch.Size([1200, 4096])
    torch.Size([5224, 1, 4096]) | torch.Size([5224])
 building dataset
    test
        torch.Size([126, 4096])
        torch.Size([300, 4096])
        torch.Size([300, 4096])
        torch.Size([280, 4096])
        torch.Size([300, 4096])
    torch.Size([1306, 1, 4096]) | torch.Size([1306])


In [ ]:
from torch.utils.data import DataLoader
dl = DataLoader(train_dataset,batch_size=4)

data = next(iter(dl))
print(data.shape)

torch.Size([4, 1, 4096])


In [ ]:
trainer = Trainer1D(
    diffusion,
    dataset = train_dataset,
    train_batch_size = 8,
    train_lr = 8e-5,
    train_num_steps = 700000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
)

In [ ]:
trainer.train()

loss: 1.1062:   0%|          | 38/700000 [00:12<64:20:00,  3.02it/s]


KeyboardInterrupt: 